# Intro 7 : Customization and Rate-Limited Concurrency

The `Session` object can be fully customized, including models, model parameters and rate limits, to accustom various usecases. 

Most usefully you can customize: 

- `service`: rate limit api service
- `llmconfig`: the default model parameters API calls in the session

By default, an `OpenAI` service will be provided with the default config settings:

```python
{
    "model": "gpt-4-turbo-preview",
    "frequency_penalty": 0,
    "max_tokens": None,
    "n": 1,
    "presence_penalty": 0, 
    "response_format": {"type": "text"}, 
    "seed": None, 
    "stop": None,
    "stream": False,
    "temperature": 0.7, 
    "top_p": 1, 
    "tools": None,
    "tool_choice": "none", 
    "user": None
    }
```
    

If you wish to change the default behavior of a session, you can:
- create a new api service
- or modify llmconfig settings

## Service

lionagi currently support `OpenAI` and `OpenRouter`, and more service options are under construction.

In [1]:
# quick start
from lionagi import Services, Session

service_openai = Services.OpenAI()

# or
service_openrouter = Services.OpenRouter()

If you want to config the service, you can consider passing in:

- `api_key`: If your API key is already stored in your environment, no explicit input is required. Otherwise, you can input your API key here.
- `key_scheme`: By default, we assume that the environment variable storing your API key named `"OPENAI_API_KEY"` (or `"OPENROUTER_API_KEY"` for `OpenRouter`). If the variable name is different, you can specify it here. 
- `token_encoding_name`: The default `token_encoding_name` is "cl100k_base".
- `max_tokens`: The maximum number of tokens allowed per interval. Default to 100000.
- `max_requests`: The maximum number of requests allowed per interval. Default to 1000.
- `interval`: The time interval in seconds for replenishing capacities. Default to 60.

Currently, we only support `chat/completions` endpoint. More endpoints will be available in the future.

## llmconfig

`llmconfig` can be set when creating the session or updated later.

In [2]:
# # example config
# config = {
#     'api_key': 'your_api_key_here',  # replace with your actual API key
#     'key_scheme': 'OPENAI_API_KEY',  # or 'OPENROUTER_API_KEY' for OpenRouter
#     'token_encoding_name': 'cl100k_base',
#     'max_tokens': 100000,
#     'max_requests': 1000,
#     'interval': 60
# }

llmconfig_ = {...}  # replace with your actual config

# passing in the llmconfig as a dict
session1 = Session("you are a helpful assistant", llmconfig=llmconfig_)

# or
session1.default_branch.llmconfig.update(llmconfig_)

## Concurrency

In [3]:
# we will use numpy to generate random numbers for this part
# %pip install numpy

In [4]:
# let us use a simple conditional calculator session as an example
# in this example, we will have two steps in the instruction, first step would be choosing between sum or diff based on a case number
# and second step would be choosing between times or plus based on the sign of the first step

system = "You are asked to perform as a calculator. Return only a numeric value, i.e. int or float, no text."

instruct1 = {
    "sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|",
}

instruct2 = {
    "diff the absolute values": "provided with 2 numbers, return the difference of absolute values. i.e. |x|-|y|",
}

instruct3 = {
    "if previous response is positive": "times 2. i.e. *2",  # case 0
    "else": "plus 2. i.e. +2",  # case 1
}

In [5]:
# create a case and context
case = 0
context = {"x": 7, "y": 3}
instruct = instruct1 if case == 0 else instruct2

In [6]:
from timeit import default_timer as timer

start = timer()
calculator = Session(system)

step1 = await calculator.chat(instruct, context=context)
step2 = await calculator.chat(
    instruct3, temperature=0.5
)  # you can also modify parameters for each API call

print(f"step1 result: {step1}")
print(f"step2 result: {step2}")

elapsed_time = timer() - start
print(f"run clock time: {elapsed_time:0.2f} seconds")

step1 result: 10
step2 result: 20
run clock time: 1.88 seconds


In [7]:
# now let us run 10 senerios in parallel
import numpy as np

num_iterations = 10

In [8]:
# generate random numbers
ints1 = np.random.randint(-10, 10, size=num_iterations)
ints2 = np.random.randint(0, 10, size=num_iterations)
cases = np.random.randint(0, 2, size=num_iterations)


contexts = []
for i in range(num_iterations):
    contexts.append({"x": str(ints1[i]), "y": str(ints2[i]), "case": str(cases[i])})

    print(contexts[i])

{'x': '-5', 'y': '9', 'case': '1'}
{'x': '-3', 'y': '2', 'case': '1'}
{'x': '-6', 'y': '0', 'case': '0'}
{'x': '-10', 'y': '7', 'case': '1'}
{'x': '-1', 'y': '0', 'case': '1'}
{'x': '7', 'y': '0', 'case': '0'}
{'x': '6', 'y': '8', 'case': '1'}
{'x': '8', 'y': '8', 'case': '0'}
{'x': '7', 'y': '8', 'case': '0'}
{'x': '7', 'y': '6', 'case': '0'}


In [9]:
async def calculator_workflow(context):

    calculator = Session(system)  # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))

    instruct = instruct1 if case == 0 else instruct2
    res1 = await calculator.chat(instruct, context=context)  # run the steps
    res2 = await calculator.chat(instruct3, temperature=0.5)

    return (res1, res2)

In [10]:
import lionagi.libs.ln_func_call as func_call

In [11]:
start = timer()
outs = await func_call.alcall(contexts, calculator_workflow)
elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 10
run clock time: 2.00 seconds


In [12]:
outs

[('4', '8'),
 ('1', '2'),
 ('6', '12'),
 ('3', '6'),
 ('1', '2'),
 ('7', '14'),
 ('-2', '0'),
 ('16', '32'),
 ('15', '30'),
 ('13', '26')]

## Customized API service concurrent calls

By default, all the session will initiate a new default service object.

If you would like to have one service using across sessions, you need to pass in the **same** service object during construction.

In [13]:
service = Services.OpenAI(max_tokens=1000, max_requests=10, interval=10)


async def calculator_workflow(context):

    calculator = Session(system, service=service)  # construct a session instance
    context = context.copy()
    case = int(context.pop("case"))
    instruct = instruct1 if case == 0 else instruct2

    res1 = await calculator.chat(instruct, context=context)  # run the steps
    res2 = await calculator.chat(instruct3)

    return (res1, res2)

In [14]:
start = timer()

outs = await func_call.alcall(contexts, calculator_workflow)

elapsed_time = timer() - start
print(f"num_workload: {num_iterations}")
print(f"run clock time: {elapsed_time:0.2f} seconds")

num_workload: 10
run clock time: 21.71 seconds


In [15]:
outs

[('4', '8'),
 ('1', '2'),
 ('6', '12'),
 ('3', '6'),
 ('1', '2'),
 ('7', '14'),
 ('2', '4'),
 ('16', '32'),
 ('15', '30'),
 ('13', '26')]